# Pima Indians Diabetes Database

### Context
This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.

### Content
The datasets consists of several medical predictor variables and one target variable, Outcome. Predictor variables includes the number of pregnancies the patient has had, their BMI, insulin level, age, and so on.

### Acknowledgements
Smith, J.W., Everhart, J.E., Dickson, W.C., Knowler, W.C., & Johannes, R.S. (1988). Using the ADAP learning algorithm to forecast the onset of diabetes mellitus. In Proceedings of the Symposium on Computer Applications and Medical Care (pp. 261--265). IEEE Computer Society Press.

### Inspiration
Can you build a machine learning model to accurately predict whether or not the patients in the dataset have diabetes or not?

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./kernel150594f677.ipynb
./diabetes.csv
./logs/fit/20200718-194236/train/events.out.tfevents.1595097760.Damians-MBP.2607.886.v2
./logs/fit/20200718-194236/train/events.out.tfevents.1595097761.Damians-MBP.profile-empty
./logs/fit/20200718-194236/train/plugins/profile/2020-07-18_19-42-41/local.trace
./logs/fit/20200718-194236/validation/events.out.tfevents.1595097761.Damians-MBP.2607.1106.v2
./.ipynb_checkpoints/kernel150594f677-checkpoint.ipynb


In [4]:
df = pd.read_csv('./diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


### Features and Labels

In [7]:
X = df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']]
y = df['Outcome']

### Training and Testing Datasets

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 0)

In [9]:
X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
279,2,108,62,10,278,25.3,0.881,22
258,1,193,50,16,375,25.9,0.655,24
249,1,111,86,19,0,30.1,0.143,23
740,11,120,80,37,150,42.3,0.785,48
725,4,112,78,40,0,39.4,0.236,38


In [10]:
y_train.head()

279    0
258    0
249    0
740    1
725    0
Name: Outcome, dtype: int64

### Classification Model Comparison

In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

h = .02  # step size in the mesh

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [12]:
scores = {}
for name,clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    scores[name] = score
print(score)

0.7844827586206896


In [14]:
sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
for i in sorted_scores:
    print(i[0], i[1])

Gaussian Process 0.8189655172413793
Decision Tree 0.8189655172413793
Linear SVM 0.8103448275862069
Naive Bayes 0.8017241379310345
Random Forest 0.7931034482758621
QDA 0.7844827586206896
AdaBoost 0.7758620689655172
Neural Net 0.7413793103448276
Nearest Neighbors 0.7241379310344828
RBF SVM 0.6724137931034483


## Neural Network - Tensorflow
https://drive.google.com/file/d/1Y42Pic0ebkxOz2PrL7hjh3Yj2vE0MVHS/view - the paper associated with this dataset achieved a higher accuracy using a Neural Network.

In [15]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [16]:
%tensorboard --logdir logs

In [17]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [18]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout, InputLayer
from tensorflow.keras import Model

In [19]:
from datetime import datetime
from packaging import version

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.0.0


In [20]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

### Define the model
Input Layer -> Fully Connected 64 -> Dropout -> Fully Connected 32 -> Dropout -> Output Layer

In [21]:
X_train.shape

(652, 8)

In [22]:
model = tf.keras.models.Sequential([
  Dense(64, input_shape=(8,), activation='elu'),
  Dropout(0.25),
  Dense(32, activation='elu'),
  Dropout(0.5),
  Dense(1, activation='sigmoid'),
])

In [23]:
loss_fn = tf.keras.losses.MeanSquaredError()

In [24]:
model.compile(optimizer='RMSProp',
              loss=loss_fn,
              metrics=['accuracy'])

In [25]:
model.fit(X_train, y_train, epochs=300, validation_split=0.1, callbacks=[tensorboard_callback])

W0725 15:20:01.097973 4606760384 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Train on 586 samples, validate on 66 samples
Epoch 1/300
586/586 [==============================] - 0s 518us/sample - loss: 0.5547 - accuracy: 0.4334 - val_loss: 0.5308 - val_accuracy: 0.4242
Epoch 2/300
586/586 [==============================] - 0s 55us/sample - loss: 0.4755 - accuracy: 0.5051 - val_loss: 0.3165 - val_accuracy: 0.6212
Epoch 3/300
586/586 [==============================] - 0s 63us/sample - loss: 0.4238 - accuracy: 0.5631 - val_loss: 0.3182 - val_accuracy: 0.6818
Epoch 4/300
586/586 [==============================] - 0s 66us/sample - loss: 0.3854 - accuracy: 0.6075 - val_loss: 0.3182 - val_accuracy: 0.6818
Epoch 5/300
586/586 [==============================] - 0s 80us/sample - loss: 0.3663 - accuracy: 0.6246 - val_loss: 0.3182 - val_accuracy: 0.6818
Epoch 6/300
586/586 [==============================] - 0s 88us/sample - loss: 0.3701 - accuracy: 0.6229 - val_loss: 0.3182 - val_accuracy: 0.6818
Epoch 7/300
586/586 [==============================] - 0s 65us/sample - loss: 

586/586 [==============================] - 0s 48us/sample - loss: 0.3605 - accuracy: 0.6365 - val_loss: 0.3182 - val_accuracy: 0.6818
Epoch 57/300
586/586 [==============================] - 0s 55us/sample - loss: 0.3535 - accuracy: 0.6451 - val_loss: 0.3182 - val_accuracy: 0.6818
Epoch 58/300
586/586 [==============================] - 0s 47us/sample - loss: 0.3559 - accuracy: 0.6433 - val_loss: 0.3182 - val_accuracy: 0.6818
Epoch 59/300
586/586 [==============================] - 0s 45us/sample - loss: 0.3637 - accuracy: 0.6348 - val_loss: 0.3182 - val_accuracy: 0.6818
Epoch 60/300
586/586 [==============================] - 0s 46us/sample - loss: 0.3593 - accuracy: 0.6399 - val_loss: 0.3182 - val_accuracy: 0.6818
Epoch 61/300
586/586 [==============================] - 0s 42us/sample - loss: 0.3637 - accuracy: 0.6348 - val_loss: 0.3182 - val_accuracy: 0.6818
Epoch 62/300
586/586 [==============================] - 0s 45us/sample - loss: 0.3524 - accuracy: 0.6451 - val_loss: 0.3182 - val_a

Epoch 112/300
586/586 [==============================] - 0s 45us/sample - loss: 0.3475 - accuracy: 0.6485 - val_loss: 0.3182 - val_accuracy: 0.6818
Epoch 113/300
586/586 [==============================] - 0s 44us/sample - loss: 0.3569 - accuracy: 0.6399 - val_loss: 0.3182 - val_accuracy: 0.6818
Epoch 114/300
586/586 [==============================] - 0s 47us/sample - loss: 0.3504 - accuracy: 0.6468 - val_loss: 0.3182 - val_accuracy: 0.6818
Epoch 115/300
586/586 [==============================] - 0s 45us/sample - loss: 0.3528 - accuracy: 0.6451 - val_loss: 0.3182 - val_accuracy: 0.6818
Epoch 116/300
586/586 [==============================] - 0s 49us/sample - loss: 0.3527 - accuracy: 0.6485 - val_loss: 0.3182 - val_accuracy: 0.6818
Epoch 117/300
586/586 [==============================] - 0s 65us/sample - loss: 0.3465 - accuracy: 0.6468 - val_loss: 0.3182 - val_accuracy: 0.6818
Epoch 118/300
586/586 [==============================] - 0s 73us/sample - loss: 0.3565 - accuracy: 0.6399 - val_

586/586 [==============================] - 0s 44us/sample - loss: 0.2988 - accuracy: 0.6741 - val_loss: 0.3331 - val_accuracy: 0.6515
Epoch 168/300
586/586 [==============================] - 0s 43us/sample - loss: 0.3413 - accuracy: 0.6485 - val_loss: 0.3493 - val_accuracy: 0.6515
Epoch 169/300
586/586 [==============================] - 0s 45us/sample - loss: 0.3347 - accuracy: 0.6399 - val_loss: 0.3668 - val_accuracy: 0.6212
Epoch 170/300
586/586 [==============================] - 0s 43us/sample - loss: 0.3485 - accuracy: 0.6229 - val_loss: 0.3635 - val_accuracy: 0.6212
Epoch 171/300
586/586 [==============================] - 0s 44us/sample - loss: 0.3416 - accuracy: 0.6382 - val_loss: 0.3516 - val_accuracy: 0.6515
Epoch 172/300
586/586 [==============================] - 0s 56us/sample - loss: 0.3211 - accuracy: 0.6570 - val_loss: 0.3653 - val_accuracy: 0.6212
Epoch 173/300
586/586 [==============================] - 0s 48us/sample - loss: 0.3217 - accuracy: 0.6570 - val_loss: 0.3359 -

586/586 [==============================] - 0s 52us/sample - loss: 0.2352 - accuracy: 0.6775 - val_loss: 0.2372 - val_accuracy: 0.6667
Epoch 223/300
586/586 [==============================] - 0s 51us/sample - loss: 0.2547 - accuracy: 0.6553 - val_loss: 0.2452 - val_accuracy: 0.6818
Epoch 224/300
586/586 [==============================] - 0s 55us/sample - loss: 0.2410 - accuracy: 0.6587 - val_loss: 0.2360 - val_accuracy: 0.6667
Epoch 225/300
586/586 [==============================] - 0s 44us/sample - loss: 0.2329 - accuracy: 0.6792 - val_loss: 0.2432 - val_accuracy: 0.6515
Epoch 226/300
586/586 [==============================] - 0s 45us/sample - loss: 0.2333 - accuracy: 0.6809 - val_loss: 0.2355 - val_accuracy: 0.6515
Epoch 227/300
586/586 [==============================] - 0s 46us/sample - loss: 0.2396 - accuracy: 0.6621 - val_loss: 0.2344 - val_accuracy: 0.6818
Epoch 228/300
586/586 [==============================] - 0s 44us/sample - loss: 0.2354 - accuracy: 0.6724 - val_loss: 0.2208 -

586/586 [==============================] - 0s 53us/sample - loss: 0.1931 - accuracy: 0.6980 - val_loss: 0.2138 - val_accuracy: 0.6818
Epoch 278/300
586/586 [==============================] - 0s 46us/sample - loss: 0.1892 - accuracy: 0.7133 - val_loss: 0.2155 - val_accuracy: 0.6970
Epoch 279/300
586/586 [==============================] - 0s 46us/sample - loss: 0.1863 - accuracy: 0.7184 - val_loss: 0.2177 - val_accuracy: 0.6667
Epoch 280/300
586/586 [==============================] - 0s 46us/sample - loss: 0.1889 - accuracy: 0.7082 - val_loss: 0.2181 - val_accuracy: 0.6667
Epoch 281/300
586/586 [==============================] - 0s 45us/sample - loss: 0.1830 - accuracy: 0.7150 - val_loss: 0.2136 - val_accuracy: 0.7121
Epoch 282/300
586/586 [==============================] - 0s 48us/sample - loss: 0.1827 - accuracy: 0.7270 - val_loss: 0.2143 - val_accuracy: 0.6970
Epoch 283/300
586/586 [==============================] - 0s 52us/sample - loss: 0.1880 - accuracy: 0.7133 - val_loss: 0.2202 -

In [26]:
results = model.evaluate(X_test, y_test)
print("test loss, test acc:", results)

W0725 15:20:17.522644 4606760384 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


116/116 [==============================] - 0s 44us/sample - loss: 0.1778 - accuracy: 0.7241
test loss, test acc: [0.1777748835497889, 0.7241379]
